# GFootball Template Bot
Below we present a simple Bot playing GFootball. The first step is to install required tools:

In [1]:
# Install:
# Kaggle environments.
!git clone https://github.com/Kaggle/kaggle-environments.git
!cd kaggle-environments && pip install .

# GFootball environment.
!apt-get update -y
!apt-get install -y libsdl2-gfx-dev libsdl2-ttf-dev

# Make sure that the Branch in git clone and in wget call matches !!
!git clone -b v2.3 https://github.com/google-research/football.git
!mkdir -p football/third_party/gfootball_engine/lib

!wget https://storage.googleapis.com/gfootball/prebuilt_gameplayfootball_v2.3.so -O football/third_party/gfootball_engine/lib/prebuilt_gameplayfootball.so
!cd football && GFOOTBALL_USE_PREBUILT_SO=1 pip3 install .

Now it is time to implement a bot. It consists of a single function **agent**, which is called by the Kaggle Environment on each step.
**Agent** receives Kaggle-specific observations as a parameter, which contain GFootball environment observations under **players_raw** key.
Detailed description of the GFootball observations is available [here](https://github.com/google-research/football/blob/master/gfootball/doc/observation.md).

In [1]:
%%writefile submission.py
from gfootball.env import football_action_set

def agent(obs):
    # Get observations for the first (and only one) player we control.
    o = obs['players_raw'][0]
    # Make sure player is running.
    if not o['sticky_actions'][8]:
        return [13]
    controlled_player_pos = o['left_team'][o['active']]
    # Does the player we control have the ball?
    if o['ball_owned_player'] == o['active'] and o['ball_owned_team'] == 0:
        # Shot if we are 'close' to the goal (based on 'x' coordinate).
        if controlled_player_pos[0] > 0.5:
            return [12]
        # Run towards the goal otherwise.
        return [5]
    else:
        # Run towards the ball.
        if o['ball'][0] > controlled_player_pos[0] + 0.05:
            return [5]
        if o['ball'][0] < controlled_player_pos[0] - 0.05:
            return [1]
        if o['ball'][1] > controlled_player_pos[1] + 0.05:
            return [7]
        if o['ball'][1] < controlled_player_pos[1] - 0.05:
            return [3]
        return [16]

In [1]:
# Set up the Environment.
from kaggle_environments import make
env = make("football", configuration={"save_video": True, "scenario_name": "11_vs_11_kaggle", "running_in_notebook": True})
env.run(["/kaggle/working/submission.py", "do_nothing"])
env.render(mode="human", width=800, height=600)

# Submit to Competition
1. "Save & Run All" (commit) this Notebook
1. Go to the notebook viewer
1. Go to "Data" section and find submission.py file.
1. Click "Submit to Competition"
1. Go to [My Submissions](https://www.kaggle.com/c/football/submissions) to view your score and episodes being played.